In [1]:
import os
import math
import itertools
import numpy as np
import pandas as pd
# PyTorch
import torch
import torchvision

In [12]:
################################################################################
def check_epochs(df, n, batch_size=128, steps=6000, drop_last=True):
    num_batches = math.floor(n/batch_size) if drop_last else math.ceil(n/batch_size)
    epochs = int(steps/num_batches)
    return df.shape[0] == epochs

def print_job(dataset, dataset_dir, experiments_dir, kappa, lr_0, method, model, 
              model_arch, n, prior_dir, prior_type, random_state, save, tune):
    model_name = f"{model}_kappa={kappa}_lr_0={lr_0}_n={n}_random_state={random_state}"
    if os.path.exists(f"{experiments_dir}/{model_name}.csv"):
        temp_df = pd.read_csv(f"{experiments_dir}/{model_name}.csv")
        n_train = n - int((1/5) * n) if tune else n
        if check_epochs(temp_df, n_train, batch_size=min(128, n_train), steps=6000, drop_last=True):
            return

    command = (
        f"python ../src/main_image_classifiers.py "
        "--batch_size=128 "
        f"--dataset=\"{dataset}\" "
        f"--dataset_dir=\"{dataset_dir}\" "
        f"--experiments_dir=\"{experiments_dir}\" "
        f"--kappa={kappa} "
        f"--lr_0={lr_0} "
        f"--method=\"{method}\" "
        f"--model=\"{model}\" "
        f"--model_arch=\"{model_arch}\" "
        f"--model_name=\"{model_name}\" "
        f"--n={n} "
        "--num_workers=0 "
        f"--prior_dir=\"{prior_dir}\" "
        f"--prior_type=\"{prior_type}\" "
        f"--random_state={random_state} "
        f"{'--save' if save else ''}"
        f"{'--tune' if tune else ''}"
    )
    
    print(f"    '{command}'")

In [8]:
model = torchvision.models.resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=10, bias=True)
print(f'd = {len(torch.nn.utils.parameters_to_vector(model.parameters()))}')
model = torchvision.models.vit_b_16()
model.heads = torch.nn.Linear(in_features=768, out_features=10, bias=True)
print(f'd = {len(torch.nn.utils.parameters_to_vector(model.parameters()))}')
model = torchvision.models.convnext_tiny()
model.classifier[2] = torch.nn.Linear(in_features=768, out_features=10, bias=True)
print(f'd = {len(torch.nn.utils.parameters_to_vector(model.parameters()))}')

d = 23528522
d = 85806346
d = 27827818


In [13]:
d = 27827818
dataset = "CIFAR-10"
dataset_dir = "/cluster/tufts/hugheslab/eharve06/CIFAR-10"
experiments_dir = "/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt-Tiny_diag_DE-ELBO"
lr_0s = [0.1, 0.01, 0.001, 0.0001]
model = "l2-sp"
method = "ELBO"
model_arch = "ConvNeXt-Tiny"
ns = [100, 1000, 10000, 50000]
prior_dir = "/cluster/tufts/hugheslab/eharve06/convnext_tiny_torchvision"
prior_type = "convnext_tiny_torchvision"
random_states = [1001, 2001, 3001]

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    print_job(dataset, dataset_dir, experiments_dir, d/n, lr_0, method, model, model_arch, n, prior_dir, prior_type, random_state, True, False)


    'python ../src/main_image_classifiers.py --batch_size=128 --dataset="CIFAR-10" --dataset_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10" --experiments_dir="/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt-Tiny_diag_DE-ELBO" --kappa=278278.18 --lr_0=0.1 --method="ELBO" --model="l2-sp" --model_arch="ConvNeXt-Tiny" --model_name="l2-sp_kappa=278278.18_lr_0=0.1_n=100_random_state=1001" --n=100 --num_workers=0 --prior_dir="/cluster/tufts/hugheslab/eharve06/convnext_tiny_torchvision" --prior_type="convnext_tiny_torchvision" --random_state=1001 --save'
    'python ../src/main_image_classifiers.py --batch_size=128 --dataset="CIFAR-10" --dataset_dir="/cluster/tufts/hugheslab/eharve06/CIFAR-10" --experiments_dir="/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt-Tiny_diag_DE-ELBO" --kappa=278278.18 --lr_0=0.1 --method="ELBO" --model="l2-sp" --model_arch="ConvNeXt-Tiny" --model_name="l2-sp_kappa=278278.

In [4]:
criterion = 'l2-zero'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'linear_probing_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10-Copy1.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10-Copy1.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [5]:
criterion = 'l2-zero'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'linear_probing_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [6]:
criterion = 'l2-sp'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa=1.0_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [7]:
criterion = 'l2-zero'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [8]:
criterion = 'l2-sp'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [8]:
kappas = torch.logspace(0, math.log(23528522), 10)
kappas

tensor([1.0000e+00, 7.6908e+01, 5.9148e+03, 4.5489e+05, 3.4985e+07, 2.6906e+09,
        2.0693e+11, 1.5914e+13, 1.2239e+15, 9.4129e+16])

In [10]:
kappas[1:]

tensor([7.6908e+01, 5.9148e+03, 4.5489e+05, 3.4985e+07, 2.6906e+09, 2.0693e+11,
        1.5914e+13, 1.2239e+15, 9.4129e+16])

In [7]:
criterion = 'l2-zero'
kappas = torch.logspace(0, math.log10(23528522), 10)
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/train_eval_comparison3'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for kappa, lr_0, n, random_state in itertools.product(kappas, lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={kappa}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={kappa} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={kappa} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --ELBo --experiments_directory='/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/train_eval_comparison3' --kappa=1.0 --lr_0=0.1 --model_name='l2-zero_kappa=1.0_lr_0=0.1_n=100_random_state=1001' --n=100 --num_workers=0 --random_state=1001 --save"
    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --ELBo --experiments_directory='/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/train_eval_comparison3' --kappa=1.0 --lr_0=0.1 --model_name='l2-zero_kappa=1.0_lr_0=0.1_n=100_random_state=2001' --n=100 --num_workers=0 --random_state=2001 --save"
    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --ELBo --experiments_directory='/c

    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --ELBo --experiments_directory='/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/train_eval_comparison3' --kappa=23528510.0 --lr_0=0.001 --model_name='l2-zero_kappa=23528510.0_lr_0=0.001_n=100_random_state=1001' --n=100 --num_workers=0 --random_state=1001 --save"
    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --ELBo --experiments_directory='/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/train_eval_comparison3' --kappa=23528510.0 --lr_0=0.001 --model_name='l2-zero_kappa=23528510.0_lr_0=0.001_n=100_random_state=2001' --n=100 --num_workers=0 --random_state=2001 --save"
    "python ../src/main_CIFAR-10-Copy3.py --batch_size=128 --criterion='l2-zero' --dataset_directory='/cluster/tufts/hugheslab/eharve06/CIFAR-10

In [6]:
criterion = 'ptyl'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_SSL_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
prior_directory = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'resnet50_ssl_prior'
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
print(count)

-1


In [9]:
criterion = 'ptyl'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --num_workers=0 --random_state={random_state} --save"')
print(count)

-1


In [10]:
criterion = 'l2-sp'
d = 85806346
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ViT_B_16_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
model_arch = 'ViT-B/16'
ns = [100, 1000, 10000, 50000]
prior_directory = '/cluster/tufts/hugheslab/eharve06/vit_b_16_torchvision'
prior_type = 'vit_b_16_torchvision'
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa=1.0_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
print(count)

-1


In [42]:
criterion = 'l2-sp'
d = 85806346
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ViT_B_16_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
model_arch = 'ViT-B/16'
ns = [100, 1000, 10000, 50000]
prior_directory = '/cluster/tufts/hugheslab/eharve06/vit_b_16_torchvision'
prior_type = 'vit_b_16_torchvision'
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
print(count)

-1


In [44]:
criterion = 'l2-sp'
d = 27827818
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt_Tiny_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
model_arch = 'ConvNeXt_Tiny'
ns = [100, 1000, 10000, 50000]
prior_directory = '/cluster/tufts/hugheslab/eharve06/convnext_tiny_torchvision'
prior_type = 'convnext_tiny_torchvision'
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa=1.0_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa=1.0 --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
print(count)

-1


In [13]:
criterion = 'l2-sp'
d = 27827818
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt_Tiny_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
model_arch = 'ConvNeXt_Tiny'
ns = [100, 1000, 10000, 50000]
prior_directory = '/cluster/tufts/hugheslab/eharve06/convnext_tiny_torchvision'
prior_type = 'convnext_tiny_torchvision'
random_states = [1001, 2001, 3001]

count = -1
for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        if not check_epochs(temp_df, n, batch_size=min(128, n), steps=6000):
            count += 1
            print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
    else:
        count += 1
        print(f'    "python ../src/main_CIFAR-10.py --batch_size=128 --criterion=\'{criterion}\' --dataset_directory=\'{dataset_directory}\' --ELBo --experiments_directory=\'{experiments_directory}\' --kappa={d/n} --lr_0={lr_0} --model_arch=\'{model_arch}\' --model_name=\'{model_name}\' --n={n} --num_workers=0 --prior_directory=\'{prior_directory}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save"')
print(count)

-1


In [14]:
criterion = 'linear_probing'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
0,linear_probing,linear_probing_kappa=235285.22_lr_0=0.1_n=100_...,100,1001,0.003776,0.5841,1.900638
1,linear_probing,linear_probing_kappa=235285.22_lr_0=0.1_n=100_...,100,2001,0.003858,0.5777,1.896756
2,linear_probing,linear_probing_kappa=235285.22_lr_0=0.1_n=100_...,100,3001,0.003716,0.5893,1.865436
3,linear_probing,linear_probing_kappa=23528.522_lr_0=0.1_n=1000...,1000,1001,0.029537,0.7346,1.054811
4,linear_probing,linear_probing_kappa=23528.522_lr_0=0.1_n=1000...,1000,2001,0.025460,0.7409,1.027795
5,linear_probing,linear_probing_kappa=23528.522_lr_0=0.1_n=1000...,1000,3001,0.022423,0.7322,1.090533
6,linear_probing,linear_probing_kappa=2352.8522_lr_0=0.1_n=1000...,10000,1001,0.303390,0.8178,0.544464
7,linear_probing,linear_probing_kappa=2352.8522_lr_0=0.1_n=1000...,10000,2001,0.330688,0.8148,0.550030
8,linear_probing,linear_probing_kappa=2352.8522_lr_0=0.1_n=1000...,10000,3001,0.319521,0.8167,0.551892
9,linear_probing,linear_probing_kappa=470.57044_lr_0=0.1_n=5000...,50000,1001,0.427061,0.8321,0.484001


In [15]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,linear_probing,100,(linear_probing_kappa=235285.22_lr_0=0.1_n=100...,"(1001, 2001, 3001)","(0.0037763251923024, 0.0038575831567868, 0.003...","(0.5841000080108643, 0.5777000188827515, 0.589...","(1.90063780412674, 1.8967560302734383, 1.86543...",0.583700,0.004744,0.5777,0.5893,1.887610,0.015759,1.865436,1.900638
1,linear_probing,1000,(linear_probing_kappa=23528.522_lr_0=0.1_n=100...,"(1001, 2001, 3001)","(0.0295371401190757, 0.0254603097692132, 0.022...","(0.7346000671386719, 0.7408999800682068, 0.732...","(1.054810880851746, 1.027794832801819, 1.09053...",0.735900,0.003669,0.7322,0.7409,1.057713,0.025695,1.027795,1.090533
2,linear_probing,10000,(linear_probing_kappa=2352.8522_lr_0=0.1_n=100...,"(1001, 2001, 3001)","(0.3033903259658814, 0.3306879689788818, 0.319...","(0.817799985408783, 0.8148000240325928, 0.8166...","(0.5444644091606139, 0.550030199241638, 0.5518...",0.816433,0.001239,0.8148,0.8178,0.548795,0.003155,0.544464,0.551892
3,linear_probing,50000,(linear_probing_kappa=470.57044_lr_0=0.1_n=500...,"(1001, 2001, 3001)","(0.4270610282735824, 0.4270989872064589, 0.427...","(0.8321000337600708, 0.8327000141143799, 0.830...","(0.484000682926178, 0.4837979791641236, 0.4834...",0.831833,0.000838,0.8307,0.8327,0.483761,0.000213,0.483483,0.484001


In [16]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,linear_probing,100,0.583700,0.5777,0.5893
1,linear_probing,1000,0.735900,0.7322,0.7409
2,linear_probing,10000,0.816433,0.8148,0.8178
3,linear_probing,50000,0.831833,0.8307,0.8327


In [17]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,linear_probing,100,1.887610,1.865436,1.900638
1,linear_probing,1000,1.057713,1.027795,1.090533
2,linear_probing,10000,0.548795,0.544464,0.551892
3,linear_probing,50000,0.483761,0.483483,0.484001


In [5]:
criterion = 'l2-zero'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI_alpha=beta'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
24,l2-zero,l2-zero_kappa=235285.22_lr_0=0.001_n=100_rando...,100,1001,3.118231,0.6037,3.183251
25,l2-zero,l2-zero_kappa=235285.22_lr_0=0.001_n=100_rando...,100,2001,3.282132,0.6339,2.507580
26,l2-zero,l2-zero_kappa=235285.22_lr_0=0.001_n=100_rando...,100,3001,2.965393,0.6300,2.582586
27,l2-zero,l2-zero_kappa=23528.522_lr_0=0.001_n=1000_rand...,1000,1001,2.142346,0.8671,0.549009
28,l2-zero,l2-zero_kappa=23528.522_lr_0=0.001_n=1000_rand...,1000,2001,2.022287,0.8718,0.542728
29,l2-zero,l2-zero_kappa=23528.522_lr_0=0.001_n=1000_rand...,1000,3001,2.089666,0.8754,0.530047
6,l2-zero,l2-zero_kappa=2352.8522_lr_0=0.1_n=10000_rando...,10000,1001,1.904582,0.9066,0.643479
31,l2-zero,l2-zero_kappa=2352.8522_lr_0=0.001_n=10000_ran...,10000,2001,1.820001,0.9145,0.265883
32,l2-zero,l2-zero_kappa=2352.8522_lr_0=0.001_n=10000_ran...,10000,3001,1.837957,0.9184,0.254586
9,l2-zero,l2-zero_kappa=470.57044_lr_0=0.1_n=50000_rando...,50000,1001,1.399720,0.9308,0.257306


In [6]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-zero,100,(l2-zero_kappa=235285.22_lr_0=0.001_n=100_rand...,"(1001, 2001, 3001)","(3.118231177330017, 3.2821323394775392, 2.9653...","(0.6037000417709351, 0.633899986743927, 0.6299...","(3.1832513648986835, 2.5075804969787603, 2.582...",0.622533,0.013412,0.6037,0.6339,2.757806,0.302390,2.507580,3.183251
1,l2-zero,1000,(l2-zero_kappa=23528.522_lr_0=0.001_n=1000_ran...,"(1001, 2001, 3001)","(2.1423457618713377, 2.0222873453140258, 2.089...","(0.8671000003814697, 0.8718000054359436, 0.875...","(0.5490089279174805, 0.5427276329040528, 0.530...",0.871433,0.003398,0.8671,0.8754,0.540595,0.007887,0.530047,0.549009
2,l2-zero,10000,(l2-zero_kappa=2352.8522_lr_0=0.1_n=10000_rand...,"(1001, 2001, 3001)","(1.904582219552994, 1.8200005746459964, 1.8379...","(0.9065999984741212, 0.9144999980926514, 0.918...","(0.6434786689758301, 0.2658828747749329, 0.254...",0.913167,0.004909,0.9066,0.9184,0.387982,0.180722,0.254586,0.643479
3,l2-zero,50000,(l2-zero_kappa=470.57044_lr_0=0.1_n=50000_rand...,"(1001, 2001, 3001)","(1.3997195198974608, 1.3903668163871767, 1.400...","(0.930799961090088, 0.928600013256073, 0.92789...","(0.2573057691574096, 0.2929177845478058, 0.283...",0.929100,0.001236,0.9279,0.9308,0.277794,0.015025,0.257306,0.292918


In [7]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-zero,100,0.622533,0.6037,0.6339
1,l2-zero,1000,0.871433,0.8671,0.8754
2,l2-zero,10000,0.913167,0.9066,0.9184
3,l2-zero,50000,0.929100,0.9279,0.9308


In [8]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-zero,100,2.757806,2.507580,3.183251
1,l2-zero,1000,0.540595,0.530047,0.549009
2,l2-zero,10000,0.387982,0.254586,0.643479
3,l2-zero,50000,0.277794,0.257306,0.292918


In [22]:
criterion = 'l2-sp'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
12,l2-sp,l2-sp_kappa=235285.22_lr_0=0.01_n=100_random_s...,100,1001,0.004855,0.7018,0.976211
13,l2-sp,l2-sp_kappa=235285.22_lr_0=0.01_n=100_random_s...,100,2001,0.004553,0.6743,1.118522
14,l2-sp,l2-sp_kappa=235285.22_lr_0=0.01_n=100_random_s...,100,3001,0.005270,0.7194,0.923916
15,l2-sp,l2-sp_kappa=23528.522_lr_0=0.01_n=1000_random_...,1000,1001,0.036803,0.8952,0.453519
16,l2-sp,l2-sp_kappa=23528.522_lr_0=0.01_n=1000_random_...,1000,2001,0.033720,0.8897,0.459936
29,l2-sp,l2-sp_kappa=23528.522_lr_0=0.001_n=1000_random...,1000,3001,0.033663,0.8687,0.415141
18,l2-sp,l2-sp_kappa=2352.8522_lr_0=0.01_n=10000_random...,10000,1001,0.048475,0.9503,0.239715
19,l2-sp,l2-sp_kappa=2352.8522_lr_0=0.01_n=10000_random...,10000,2001,0.054215,0.9498,0.242924
20,l2-sp,l2-sp_kappa=2352.8522_lr_0=0.01_n=10000_random...,10000,3001,0.054031,0.9519,0.229334
21,l2-sp,l2-sp_kappa=470.57044_lr_0=0.01_n=50000_random...,50000,1001,0.053313,0.9678,0.116133


In [23]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,100,(l2-sp_kappa=235285.22_lr_0=0.01_n=100_random_...,"(1001, 2001, 3001)","(0.0048551999730989, 0.0045528121991083, 0.005...","(0.7017999887466431, 0.674299955368042, 0.7193...","(0.9762106119155888, 1.1185215423583987, 0.923...",0.698500,0.018559,0.6743,0.7194,1.006216,0.082232,0.923916,1.118522
1,l2-sp,1000,(l2-sp_kappa=23528.522_lr_0=0.01_n=1000_random...,"(1001, 2001, 3001)","(0.036803496042639, 0.0337195919502526, 0.0336...","(0.8952000141143799, 0.8896999359130859, 0.868...","(0.4535194923877715, 0.4599358924865723, 0.415...",0.884533,0.011419,0.8687,0.8952,0.442865,0.019779,0.415141,0.459936
2,l2-sp,10000,(l2-sp_kappa=2352.8522_lr_0=0.01_n=10000_rando...,"(1001, 2001, 3001)","(0.0484747754009068, 0.054214593064934, 0.0540...","(0.9502999782562256, 0.9498000144958496, 0.951...","(0.2397154236674309, 0.2429243560791015, 0.229...",0.950667,0.000896,0.9498,0.9519,0.237325,0.005800,0.229334,0.242924
3,l2-sp,50000,(l2-sp_kappa=470.57044_lr_0=0.01_n=50000_rando...,"(1001, 2001, 3001)","(0.0533132338329553, 0.0527807938820719, 0.054...","(0.9678000211715698, 0.969200074672699, 0.9671...","(0.1161334464430809, 0.1156534536063671, 0.120...",0.968067,0.000838,0.9672,0.9692,0.117494,0.002272,0.115653,0.120695


In [24]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,100,0.698500,0.6743,0.7194
1,l2-sp,1000,0.884533,0.8687,0.8952
2,l2-sp,10000,0.950667,0.9498,0.9519
3,l2-sp,50000,0.968067,0.9672,0.9692


In [25]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,100,1.006216,0.923916,1.118522
1,l2-sp,1000,0.442865,0.415141,0.459936
2,l2-sp,10000,0.237325,0.229334,0.242924
3,l2-sp,50000,0.117494,0.115653,0.120695


In [26]:
criterion = 'ptyl'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_SSL_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
24,ptyl,ptyl_kappa=235285.22_lr_0=0.001_n=100_random_s...,100,1001,0.071608,0.5605,1.431721
25,ptyl,ptyl_kappa=235285.22_lr_0=0.001_n=100_random_s...,100,2001,0.052211,0.5779,1.329025
26,ptyl,ptyl_kappa=235285.22_lr_0=0.001_n=100_random_s...,100,3001,0.055849,0.5856,1.327916
27,ptyl,ptyl_kappa=23528.522_lr_0=0.001_n=1000_random_...,1000,1001,0.062456,0.7828,0.744150
28,ptyl,ptyl_kappa=23528.522_lr_0=0.001_n=1000_random_...,1000,2001,0.058029,0.7777,0.775563
29,ptyl,ptyl_kappa=23528.522_lr_0=0.001_n=1000_random_...,1000,3001,0.046580,0.7901,0.757146
30,ptyl,ptyl_kappa=2352.8522_lr_0=0.001_n=10000_random...,10000,1001,0.278554,0.9006,0.306169
31,ptyl,ptyl_kappa=2352.8522_lr_0=0.001_n=10000_random...,10000,2001,0.267516,0.9084,0.289611
32,ptyl,ptyl_kappa=2352.8522_lr_0=0.001_n=10000_random...,10000,3001,0.249591,0.9082,0.284399
21,ptyl,ptyl_kappa=470.57044_lr_0=0.01_n=50000_random_...,50000,1001,0.299012,0.9663,0.116440


In [27]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,ptyl,100,(ptyl_kappa=235285.22_lr_0=0.001_n=100_random_...,"(1001, 2001, 3001)","(0.0716077564284205, 0.0522108107805252, 0.055...","(0.5605000257492065, 0.5778999924659729, 0.585...","(1.4317212291717527, 1.3290254764556884, 1.327...",0.574667,0.010499,0.5605,0.5856,1.362888,0.048675,1.327916,1.431721
1,ptyl,1000,(ptyl_kappa=23528.522_lr_0=0.001_n=1000_random...,"(1001, 2001, 3001)","(0.0624558735460043, 0.0580292544156312, 0.046...","(0.782800018787384, 0.7776999473571777, 0.7900...","(0.7441499309539797, 0.7755626309394835, 0.757...",0.783533,0.005089,0.7777,0.7901,0.758953,0.012888,0.744150,0.775563
2,ptyl,10000,(ptyl_kappa=2352.8522_lr_0=0.001_n=10000_rando...,"(1001, 2001, 3001)","(0.2785537813949585, 0.2675162135386467, 0.249...","(0.9006000757217407, 0.908400058746338, 0.9082...","(0.3061692847251892, 0.289611121225357, 0.2843...",0.905733,0.003631,0.9006,0.9084,0.293393,0.009281,0.284399,0.306169
3,ptyl,50000,(ptyl_kappa=470.57044_lr_0=0.01_n=50000_random...,"(1001, 2001, 3001)","(0.2990124519071578, 0.2682621651830673, 0.272...","(0.9663000106811525, 0.9651000499725342, 0.966...","(0.1164396757483482, 0.1157223566055297, 0.118...",0.965933,0.000591,0.9651,0.9664,0.117001,0.001333,0.115722,0.118840


In [28]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,ptyl,100,0.574667,0.5605,0.5856
1,ptyl,1000,0.783533,0.7777,0.7901
2,ptyl,10000,0.905733,0.9006,0.9084
3,ptyl,50000,0.965933,0.9651,0.9664


In [29]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,ptyl,100,1.362888,1.327916,1.431721
1,ptyl,1000,0.758953,0.744150,0.775563
2,ptyl,10000,0.293393,0.284399,0.306169
3,ptyl,50000,0.117001,0.115722,0.118840


In [30]:
criterion = 'ptyl'
d = 23528522
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
12,ptyl,ptyl_kappa=235285.22_lr_0=0.01_n=100_random_st...,100,1001,0.005751,0.6988,0.992071
13,ptyl,ptyl_kappa=235285.22_lr_0=0.01_n=100_random_st...,100,2001,0.005645,0.6787,1.082820
14,ptyl,ptyl_kappa=235285.22_lr_0=0.01_n=100_random_st...,100,3001,0.005737,0.7213,0.919453
15,ptyl,ptyl_kappa=23528.522_lr_0=0.01_n=1000_random_s...,1000,1001,0.032229,0.8946,0.457152
16,ptyl,ptyl_kappa=23528.522_lr_0=0.01_n=1000_random_s...,1000,2001,0.034424,0.8905,0.458295
17,ptyl,ptyl_kappa=23528.522_lr_0=0.01_n=1000_random_s...,1000,3001,0.031471,0.8897,0.468716
18,ptyl,ptyl_kappa=2352.8522_lr_0=0.01_n=10000_random_...,10000,1001,0.050434,0.9515,0.230563
19,ptyl,ptyl_kappa=2352.8522_lr_0=0.01_n=10000_random_...,10000,2001,0.054827,0.9486,0.238886
20,ptyl,ptyl_kappa=2352.8522_lr_0=0.01_n=10000_random_...,10000,3001,0.053040,0.9541,0.216595
21,ptyl,ptyl_kappa=470.57044_lr_0=0.01_n=50000_random_...,50000,1001,0.054194,0.9698,0.120934


In [31]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,ptyl,100,(ptyl_kappa=235285.22_lr_0=0.01_n=100_random_s...,"(1001, 2001, 3001)","(0.0057513121049851, 0.0056449412135407, 0.005...","(0.6987999677658081, 0.6786999702453613, 0.721...","(0.9920711883544922, 1.0828201686859134, 0.919...",0.699600,0.017401,0.6787,0.7213,0.998115,0.066831,0.919453,1.082820
1,ptyl,1000,(ptyl_kappa=23528.522_lr_0=0.01_n=1000_random_...,"(1001, 2001, 3001)","(0.0322288798421621, 0.0344244828503578, 0.031...","(0.894599974155426, 0.8904999494552612, 0.8896...","(0.4571515614509582, 0.4582948702812194, 0.468...",0.891600,0.002146,0.8897,0.8946,0.461388,0.005203,0.457152,0.468716
2,ptyl,10000,(ptyl_kappa=2352.8522_lr_0=0.01_n=10000_random...,"(1001, 2001, 3001)","(0.0504343263658881, 0.0548270693182945, 0.053...","(0.9514999985694884, 0.9485999941825868, 0.954...","(0.2305629624605178, 0.2388862297058105, 0.216...",0.951400,0.002247,0.9486,0.9541,0.228681,0.009197,0.216595,0.238886
3,ptyl,50000,(ptyl_kappa=470.57044_lr_0=0.01_n=50000_random...,"(1001, 2001, 3001)","(0.0541938937689662, 0.0525872783481478, 0.055...","(0.969800055027008, 0.9682999849319458, 0.9675...","(0.1209341276764869, 0.1162585879325866, 0.118...",0.968533,0.000953,0.9675,0.9698,0.118674,0.001912,0.116259,0.120934


In [32]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,ptyl,100,0.699600,0.6787,0.7213
1,ptyl,1000,0.891600,0.8897,0.8946
2,ptyl,10000,0.951400,0.9486,0.9541
3,ptyl,50000,0.968533,0.9675,0.9698


In [33]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,ptyl,100,0.998115,0.919453,1.082820
1,ptyl,1000,0.461388,0.457152,0.468716
2,ptyl,10000,0.228681,0.216595,0.238886
3,ptyl,50000,0.118674,0.116259,0.120934


In [8]:
criterion = 'l2-sp'
d = 85806346
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ViT_B_16_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa={d/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
12,l2-sp,l2-sp_kappa=858063.46_lr_0=0.01_n=100_random_s...,100,1001,0.001290,0.8975,0.323650
13,l2-sp,l2-sp_kappa=858063.46_lr_0=0.01_n=100_random_s...,100,2001,0.001280,0.8705,0.411592
14,l2-sp,l2-sp_kappa=858063.46_lr_0=0.01_n=100_random_s...,100,3001,0.001307,0.8872,0.354825
15,l2-sp,l2-sp_kappa=85806.346_lr_0=0.01_n=1000_random_...,1000,1001,0.001219,0.9415,0.241270
16,l2-sp,l2-sp_kappa=85806.346_lr_0=0.01_n=1000_random_...,1000,2001,0.001232,0.9421,0.229562
17,l2-sp,l2-sp_kappa=85806.346_lr_0=0.01_n=1000_random_...,1000,3001,0.001212,0.9400,0.256549
18,l2-sp,l2-sp_kappa=8580.6346_lr_0=0.01_n=10000_random...,10000,1001,0.002414,0.9757,0.120763
19,l2-sp,l2-sp_kappa=8580.6346_lr_0=0.01_n=10000_random...,10000,2001,0.002367,0.9730,0.126796
20,l2-sp,l2-sp_kappa=8580.6346_lr_0=0.01_n=10000_random...,10000,3001,0.002391,0.9731,0.126974
21,l2-sp,l2-sp_kappa=1716.12692_lr_0=0.01_n=50000_rando...,50000,1001,0.007230,0.9830,0.067067


In [46]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,100,(l2-sp_kappa=858063.46_lr_0=0.01_n=100_random_...,"(1001, 2001, 3001)","(0.0012898166198283, 0.001280097302515, 0.0013...","(0.8974999785423279, 0.8705000281333923, 0.887...","(0.3236502715587615, 0.4115915064811708, 0.354...",0.885067,0.011125,0.8705,0.8975,0.363356,0.036405,0.323650,0.411592
1,l2-sp,1000,(l2-sp_kappa=85806.346_lr_0=0.01_n=1000_random...,"(1001, 2001, 3001)","(0.0012192867681384, 0.001232363811694, 0.0012...","(0.9415000081062316, 0.9420999884605408, 0.940...","(0.2412695629358292, 0.2295618210792542, 0.256...",0.941200,0.000883,0.9400,0.9421,0.242460,0.011049,0.229562,0.256549
2,l2-sp,10000,(l2-sp_kappa=8580.6346_lr_0=0.01_n=10000_rando...,"(1001, 2001, 3001)","(0.0024139546461403, 0.0023668793648108, 0.002...","(0.9757000207901, 0.9729999303817748, 0.973099...","(0.1207628115473315, 0.1267956589072943, 0.126...",0.973933,0.001250,0.9730,0.9757,0.124844,0.002887,0.120763,0.126974
3,l2-sp,50000,(l2-sp_kappa=1716.12692_lr_0=0.01_n=50000_rand...,"(1001, 2001, 3001)","(0.007229988314867, 0.0071353351466804, 0.0070...","(0.9829999804496764, 0.9812999367713928, 0.981...","(0.067066892581433, 0.0721231608636677, 0.0737...",0.982000,0.000726,0.9813,0.9830,0.070978,0.002843,0.067067,0.073743


In [47]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,100,0.885067,0.8705,0.8975
1,l2-sp,1000,0.941200,0.9400,0.9421
2,l2-sp,10000,0.973933,0.9730,0.9757
3,l2-sp,50000,0.982000,0.9813,0.9830


In [48]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,100,0.363356,0.323650,0.411592
1,l2-sp,1000,0.242460,0.229562,0.256549
2,l2-sp,10000,0.124844,0.120763,0.126974
3,l2-sp,50000,0.070978,0.067067,0.073743


In [10]:
criterion = 'l2-sp'
d = 27827818
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_ConvNeXt_Tiny_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_loss', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_kappa=1.0_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_loss'].idxmin()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_loss,test_acc,test_nll
12,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=100_random_state=1001,100,1001,2.302246,0.6954,2.296828
13,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=100_random_state=2001,100,2001,2.300187,0.6943,2.296921
38,l2-sp,l2-sp_kappa=1.0_lr_0=0.0001_n=100_random_state...,100,3001,2.302801,0.7048,2.302463
15,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=1000_random_state=...,1000,1001,2.240487,0.6065,2.094569
16,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=1000_random_state=...,1000,2001,2.232666,0.5601,2.082709
17,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=1000_random_state=...,1000,3001,2.233019,0.5385,2.078624
18,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=10000_random_state...,10000,1001,1.110588,0.9139,0.348263
19,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=10000_random_state...,10000,2001,1.111894,0.9147,0.347997
20,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=10000_random_state...,10000,3001,1.108950,0.9147,0.349764
21,l2-sp,l2-sp_kappa=1.0_lr_0=0.01_n=50000_random_state...,50000,1001,0.493595,0.9541,0.139473


In [11]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_loss,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,100,(l2-sp_kappa=1.0_lr_0=0.01_n=100_random_state=...,"(1001, 2001, 3001)","(2.302246141433716, 2.30018720626831, 2.302800...","(0.695400059223175, 0.6942999958992004, 0.7048...","(2.296827983474732, 2.296920763778687, 2.30246...",0.698167,0.004712,0.6943,0.7048,2.298737,0.002635,2.296828,2.302463
1,l2-sp,1000,(l2-sp_kappa=1.0_lr_0=0.01_n=1000_random_state...,"(1001, 2001, 3001)","(2.2404872352600096, 2.2326656087875367, 2.233...","(0.6065000295639038, 0.5601000189781189, 0.538...","(2.094568626785278, 2.082708916854857, 2.07862...",0.568367,0.028370,0.5385,0.6065,2.085301,0.006762,2.078624,2.094569
2,l2-sp,10000,(l2-sp_kappa=1.0_lr_0=0.01_n=10000_random_stat...,"(1001, 2001, 3001)","(1.110588353881836, 1.1118941320323945, 1.1089...","(0.9138999581336976, 0.9146999716758728, 0.914...","(0.3482629978656767, 0.3479969788074494, 0.349...",0.914433,0.000377,0.9139,0.9147,0.348675,0.000778,0.347997,0.349764
3,l2-sp,50000,(l2-sp_kappa=1.0_lr_0=0.01_n=50000_random_stat...,"(1001, 2001, 3001)","(0.4935954007759094, 0.4926021788511276, 0.493...","(0.9540999531745912, 0.9545000791549684, 0.955...","(0.1394728645920753, 0.1388771866559982, 0.140...",0.954600,0.000455,0.9541,0.9552,0.139595,0.000642,0.138877,0.140434


In [12]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,100,0.698167,0.6943,0.7048
1,l2-sp,1000,0.568367,0.5385,0.6065
2,l2-sp,10000,0.914433,0.9139,0.9147
3,l2-sp,50000,0.954600,0.9541,0.9552


In [13]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,100,2.298737,2.296828,2.302463
1,l2-sp,1000,2.085301,2.078624,2.094569
2,l2-sp,10000,0.348675,0.347997,0.349764
3,l2-sp,50000,0.139595,0.138877,0.140434
